In [ ]:
# When R2 is negative, Model is performing worse than the mena.

# Need to Add ability to filter ML Models Based on Dataset size.
# Hard Code Small, Medium Large, 
# Add new columns and helpful hitns to Datasheet

In [ ]:
## As Part of ML Process Need to Develop a specific plan, including Reviewing Documentation, Updating DOcumentation 

In [1]:
import numpy as np

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import time
import sys

sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

from MLPipeline import MLPipelineSample
#CalculateRegressionPerformance,SKLearnModelList,apply_scaling
#from sklearn.model_selection import train_test_split

from DataSets import iris_df,diabetes_df

df = iris_df.copy()
df1 = diabetes_df.copy()

results_df = MLPipelineSample(df1,
                              scaler='normal',
                              target_column='Target',
                              sample_override=1,
                              run_all_size_models=1,
                              test_size=0.2)



In [ ]:

from DataSets import MNIST_SKLEARN
from MLPipeline import MLPipelineSample

df,z = MNIST_SKLEARN(normalize=False,flatten=True,return_value='df')

results_df = MLPipelineSample(df,
                              scaler='normal',
                              ml_model_type='classifier',
                              target_column='Target',
                              test_size=0.2)
results_df
